<p><b><center>Investigating Smooth-Particle Hydrodynamics (SPH) with Python</center></b></p>

<u>Table of Contents</u>
1. Introduction
2. Modelling
3. The $n$-body model
4. The dam break model
5. Conclusion
6. References
7. Appendices

# Introduction
## Fluid dynamics: why bother?
[Fluid dynamics](https://en.wikipedia.org/wiki/Fluid_dynamics) is, broadly speaking, the study of how fluids flow. Understanding how fluids flow is important for many specializations in engineering. Here are some interesting applications of fluid dynamics:
1. [Biomedical Engineering](https://pubmed.ncbi.nlm.nih.gov/34476751/): the design of medical equipment such as vascular stents relies on a good understanding of how blood flows in the body.
2. [Disaster Protection](https://link.springer.com/chapter/10.1007/978-3-211-69356-8_1): tsunami forecasting and risk evaluation requires a good understanding of how the oceans move.
3. [Industrial Design](https://www.dynamiq-eng.co.uk/pages/Fluid_mechanics_dynamics.php): building safe, efficient, and functioning equipment needs a good understanding of how "fluids" such as heat and pressure change over time.
4. [Astrophysics](https://arxiv.org/abs/1109.2219): somewhat surprisingly, we can model space as a fluid as well.

## Fluid dynamics: how to implement?
As with anything in science, we first begin our study of fluid dynamics by learning how to model it. Fluids are typically modelled as _continuous_, meaning that we treat them as a continuous mass, rather than a set of discrete particles. Consider a mole of water - is it easier to think of it as $18ml$ of water or $\sim 6\times10^{23}$ water molecules interacting with each other?

In our simplified models, these bodies of water then obey several assumptions relating quantities such as their pressure, density, volume, and mass. We briefly list some assumptions, taken from [Wikipedia](https://en.wikipedia.org/wiki/Fluid_mechanics):
1. Conservation of mass
2. Conservation of energy
3. Conservation of momentum
4. The continuum assumption

We can explain what these assumptions mean using a simple bowl of water taken from [Shutterstock](https://www.shutterstock.com/image-vector/water-bowl-illustration-vector-cartoon-260nw-2210641359.jpg).

![](https://www.shutterstock.com/image-vector/water-bowl-illustration-vector-cartoon-260nw-2210641359.jpg)

**Conservation of mass**: the amount of water in the bowl will not change (unless we pour some water out, or pour some water in).

**Conservation of energy**: the energy of the water in the bowl will not change (assuming the bowl is an inch thick shell of $Bi_4O_4SeCl_2$, the [worst known heat conductor in the world](https://www.sciencealert.com/sustainability-win-scientists-make-material-with-the-lowest-thermal-conductivity-yet), preventing the heat energy of the water from moving to its surroundings).

**Conservation of momentum**: the total momentum of the water remains constant, assuming that the bowl is a closed system.

**The continuum assumption**: the body of water is a continuous distribution of matter, where properties such as pressure ["may vary point-by-point."](http://pillars.che.pitt.edu/student/slide.cgi?course_id=10&slide_id=5.0) A better example than our bowl of water would the ocean - the temperature of the water on the beaches of Delhi is different from the temperature of the water on the shores of Normandy, even though the oceans can be thought of as a continuous distribution of water.

With these assumptions, we can then proceed to model the various properties of the fluid using a simple set of equations. [NASA](https://www.grc.nasa.gov/www/k-12/airplane/nseqs.html) has provided a helpful graphic showing the equations we use:

![](https://www.grc.nasa.gov/www/k-12/airplane/Images/nseqs.gif)

Oh.

## Practical fluid dynamics: challenges
If we are to follow the equations in the NASA graphic, we are going to have one hell of a time trying to solve anything. Solving a single partial differential equation (PDE) is tough enough on its own, and solving several linked PDEs is very very hard. If we are to try to solve it by hand, we quickly realize that doing so is an [open problem with a bounty of a million dollars](https://en.wikipedia.org/wiki/Navier–Stokes_existence_and_smoothness).

The challenges are obvious, and seemingly insurmountable. Despite this, engineers and scientists have been successfully surmounting them for decades. How?

## Practical fluid dynamics: approaches
Enter [computational fluid dynamics](https://www.simscale.com/docs/simwiki/cfd-computational-fluid-dynamics/what-is-cfd-computational-fluid-dynamics/), or CFD for short. To explain CFD, we briefly return to secondary school calculus. We recall the geometric intuition behind differentiation - differentiating our function gives us the rate of change of the function, and partial differential equations are nothing but lots of differentiation.

In an ideal world, the derivative $f'(x)$ of a function $f(x)$ is given by the equation $$f'(x)=\lim_{h\to 0}\frac{f(x+h)-f(x)}{h}.$$ In our unideal world, we can set the value of $h$ to $0.005$ (an arbitrary number), and declare that to be the rate of change over time. In other words, we set our time step to $0.005$ seconds (or any other sufficiently small number that you like), and use this to approximate the complicated equations we see above. Informally, we say that we can approximate $f'(x)$ with the following: $$f'(x) \approx \frac{f(x+0.005)-f(x)}{0.005}.$$

This converts the problem from a calculus problem with differentiation and integration to a computational problem, involving millions of addition, subtraction, multiplication, and division operations. To a computer, the latter is much simpler to perform than the former.

This allows us to introduce smooth-particle hydrodynamics (SPH), a computational method that translates the fancy equations to a much uglier, but simpler to compute version.

## SPH: an introduction
The very first thing done in SPH is to take a closer look at the continuum assumption made above. Recall that the continuum assumption for a fluid says that properties such as density and pressure of the fluid may vary throughout the fluid. SPH takes that to an extreme - we can therefore model the fluid as a large collection of particles, each particle having its own properties such as density and pressure.

Now that we have a very large collection of particles, we need to define the interactions between these particles. As Dan Price [writes](https://arxiv.org/abs/1111.1259),

> SPH starts with the answer to the question: "How do I compute a density given a distribution of point-mass particles?"

Recall that density of an object is given by the mass of the object divided by its volume; in other words, we can think of density as how much "stuff" there is in a fixed part of space. In SPH, we assume that the mass of each of our particles is constant. Therefore, the only thing that can affect the density of a particle is the amount of space it occupies.

How then, do we measure this amount of space? Here comes the tricky part! Intuitively, a particle cannot have a changing volume (think about it: do we expect the volume of a water molecule to change over time? No!). However, under the assumptions in SPH, **a particle is defined as a fixed amount of mass**. In other words, **particles do not represent individual molecules**. In reality, each SPH "particle" is a stand-in for something like 10,000 water molecules.

(Remark: the 10,000 figure given above is _wildly_ inaccurate. A mole of water has $\sim 6\times10^{23}$ water molecules, and if we represent only 10,000 water molecules with a particle, we will require _six quintillion_ SPH particles to simulate 18ml of water. It is likely that an SPH particle is a stand in for several gorjillion particles, allowing us to model our fluid systems with a small enough number of particles that our computers will not explode.)

With the knowledge that each SPH particle is actually several gorjillion water molecules, this allows us to introduce the notion of _density_ for an SPH particle. Several gorjillion water molecules are obviously not a point mass, and will occupy a certain amount of space, hence giving us the density of the SPH particle.

## SPH: a practical view
Now that we've introduced the notion of SPH particles, what exactly can we do with them? The short answer is that we can use these SPH particles to model how the fluid changes over time. SPH allows us to derive the equations of motion for each SPH particle given the positions and density of other SPH particles, through the following equation: $$\frac{d\mathbf{v}_i}{dt}=-\sum_jm_j\left(\frac{P_i}{\rho_i^2}+\frac{P_j}{\rho_j^2}\right)\nabla W(|\mathbf{r}_i-\mathbf{r}_j|,h),$$ where $\frac{d\mathbf{v}_i}{dt}$ is the acceleration of the $i$th particle, $m_j$ is the mass of the $j$th particle, $P_i$ and $P_j$ are the pressures acting on the $i$th and $j$th particles, $\rho_i$ and $\rho_j$ are the densities of the $i$th and $j$th particles, and $\nabla W(|\mathbf{r}_i-\mathbf{r}_j|,h)$ is a smoothing function, which we will explain shortly.

That was a whole chunk of mathematics, but **the key takeaway is that we can compute the acceleration of an SPH particle through a lot of simple mathematical operations (addition, subtraction, division, multiplication), without solving any differential equations.**

### The smoothing function $\nabla W(|\mathbf{r}_i-\mathbf{r}_j|,h)$
The smoothing function is a fancy mathematical way that the particles which are closer to each other have a greater effect on each other. The [gaussian kernel](https://pages.stat.wisc.edu/~mchung/teaching/MIA/reading/diffusion.gaussian.kernel.pdf.pdf) is a commonly used smoothing function that is shaped like a bell curve, with the height of the graph representing the effect of the particles surrounding it.

![](http://www.imagemet.com/WebHelp6/Content/Images/Gauss3D.png)

To use the gaussian kernel, we pick the $i$th particle of the fluid to study, and then we compare its distance to every other particle in the fluid (represented by $|\mathbf{r}_i-\mathbf{r}_j|$), which denotes the effect that the $j$th particle has on the $ith$ particle.

If the $j$th particle is _very, very_ far away, we see from the image that its effect is basically reduced to $0$. However, if the $j$th particle is _very, very_ close, we see that its effect is still relatively strong.

To summarize, the smoothing function is nothing but a fancy way of saying that particles close to each other have a greater effect on each other than particles which are far apart.

## Moving forward
Now that we have a surface understanding of SPH and how it works, lets proceed to implementing it as code.

# Modelling Fluid Dynamics
## Planning the code
Before we jump right into writing code, let's pause and think about _what_ exactly we want from our code. We want to be able to simulate particles with the following properties:
1. A fixed mass
2. A position in space (i.e., 3D coordinates)
3. A direction vector (i.e., another set of 3D coordinates)
4. A density
5. A pressure

Beyond that, we also need to find a way to make these particles interact with each other, so we can perform the calculations required by SPH. Thankfully, we can write out each of these interactions inside a _function_, allowing us to simplify our code. Some of the functions we might require include:
1. A function to calculate the distance between two particles.
2. A function to update the position of a particle in space, given a certain timestep.
3. A function to calculate the change in density of a particle, given the position of the neighbouring particles.

In short, what we require from our code is (a) the ability to store lots of particles, and (b) the ability to work with particles.

## Version 1.1: Python lists
We start with a naive attempt at turning our particles into code. We can represent our particles as a list in Python:

```
testParticle1 = [mass, density, pressure, xCoord, yCoord, zCoord, xDir, yDir, zDir]
```

Writing our functions to deal with such particles will involve lots of list comprehension!

## Version 1.2: Numpy arrays
However, we quickly notice that all these variables are actually of the `float` data type, which means that we can also store the data of the particle within a `numpy` array. This will greatly involve the speed of our calculations, as `numpy` arrays are optimized for dealing with numerical computations. Lets try and write some pseudocode for only five particles. **Do not run the next code block, it will throw an error!**

In [ ]:
# Do NOT run this code block - it will throw errors!
import numpy as np
from matplotlib import pyplot as plt

# Creates our array of particles
particles = []
for i in range(5):
    particles.append(np.array(["""Particle data here!"""]))
    
# Implementing the fluid equations
timestep = 0.01 # Sets the time step
def update_particles(particles, timestep):
    """
    Complex mathematical equations here!
    This function will update the particle positions and
    items in our list of particles.
    """

# Running the simulation
start_time = 0
end_time = 10 # A 10 second simulation!
particle_position_data = [] # To store the particle positions at each time step

while (start_time < end_time):
    particle_position_data.append(["""Extract the data using list comprehension"""])
    update_particles(particles, timestep)
    start_time += timestep
    
# Plotting the particles
ax = plt.axes(projection='3d')
for i in range(5):
    # All data is taken from particle_position_data
    x = """X coordinates of one particle over time"""
    y = """Y coordinates of one particle over time"""
    z = """Z coordinates of one particle over time"""
    ax.plot3D(x, y, z)
plt.show()

The above pseudocode provides a framework for our SPH simulation. We can turn it into a series of steps for modelling _any_ SPH simulation:
1. Initialize our particles.
2. Define our time step and our functions to update the particle data.
3. Run the simulation, storing the position data of the particles at each time step.
4. Use `matplotlib` to plot the movement of the particles over time.

## Attempt 2: New data types
Although the above code is perfectly functional, it is not exactly the prettiest either. If our particles are simply lists of floats, it is very hard for us to remember what exactly it is - is index 5 supposed to be the y-coordinate of the particle? Or is it referring to the density of the particle?

We therefore consider the idea of introducing a _new_ data type - a `Particle` data type. Just as how we can have variables which are strings, integers, dictionaries and so on, we can also have a variable which is a `Particle`.

One might wonder - why should we create a new data type? How can we create a new data type? To answer these questions, we can think about it from the perspective of the requirements of the program. For example, in a calculator program, we may frequently work with decimal numbers, and we'll therefore need our variables to be able to store floats. In our SPH simulations, we'll frequently be working with particles, and we therefore need our variables to store the attributes of a particle.

Thankfully, Python allows us to construct new data types, known as _classes_. Constructing a new data type means defining a set of functions to interact with that data type. (For example, the addition `+` operation can work with many of Python's default data types, such as integers and strings and floats). This gives us a great deal of flexibility in deciding what exactly we can do with the data types. In the example below, we define a new data type called `ExampleParticle`, and give a few simple examples of how we can interact with it.

In [1]:
class ExampleParticle():
    """
    A simple particle class for demonstrating
    classes. Particles are assumed to have constant
    mass, and only have a position and velocity.
    ...
    Attributes
    ----------
    p : list
        List with three elements, representing the
        particle's position in 3D space.
        
    d : list
        List with three elements, representing the
        particle's direction vector in 3D space.

    Methods
    -------
    getPos()
        Returns position of the particle.
        
    getDir()
        Returns direction vector of the particle.

    getMagnitude()
        Returns magnitude of the direction vector.
        
    setPos(newPos)
        Updates position of the particle.
        
    setDir(newDir)
        Updates direction vector of the particle.
    """

    def __init__(self, position, direction):
        self._p = position
        self._d = direction
    
    def __str__(self):
        return f"Current position: {self._p}. Current direction: {self._d}."
    
    def getPos(self):
        return self._p
    
    def getDir(self):
        return self._d

    def getMagnitude(self):
        return np.linalg.norm(self._p)
    
    def setPos(self, newPos):
        self._p = newPos
        
    def setDir(self, newDir):
        self._d = newDir

In [4]:
# Initializing a new ExampleParticle with
# position (0, 0, 0) and direction vector
# (1, -2, 3).
exampleParticle1 = ExampleParticle([0, 0, 0], [1, -2, 3])

print(exampleParticle1)

# Changing the position of the particle to
# (3, 5, -1).
exampleParticle1.setPos([3, 5, -1])

print(exampleParticle1)
print(exampleParticle1.getPos())

Current position: [0, 0, 0]. Current direction: [1, -2, 3].
Current position: [3, 5, -1]. Current direction: [1, -2, 3].
[3, 5, -1]


We can now proceed to construct more complex classes to model our particles, taking into account factors such as mass and density. The later sections will further elaborate on the construction of new classes to model particles and physical systems in Python.

# The $n$-body problem
Before proceeding to model fluids, we'll start by checking out the [$n$-body problem](https://en.wikipedia.org/wiki/N-body_problem), a classic demonstration of the style of computation demanded by SPH. The $n$-body problem has a relatively simple statement: given a bunch of particles which interact with each other through the gravitational force of attraction, predict their movement in 3D space over time. The subsequent code block will walk us through the framework described above.

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt

# 1. Initializing the particles
particles=np.array([[1,0.3,0.4,0.3,0.1,0.6,0.1],
                    [1,0.4,0.7,0.1,0.8,-0.9,-0.6],
                   ])

# 2. Define the functions

def update_particles(particles,t): # Updates the coordinates and velocities of our particles
    
    G = 1
    n = len(particles)
    mass = np.array([particles[i, 0] for i in range(n)])
    coordinates = np.array([particles[i, 1:4] for i in range(n)])
    velocity = np.array([particles[i, 4:7] for i in range(n)])
    
    # Helper function to calculate the acceleration between the particles m and n.
    def acceleration_between_2_points(m,n):
        return mass[m]*(coordinates[n]-coordinates[m])/(np.sqrt(sum((coordinates[m]-coordinates[n])**2)))**3

    # Helper function to calculate the acceleration for
    # particle m taking all other particles into consideration.
    def get_acceleration(m):
        other_indices = list(range(n))
        other_indices.remove(m)
        acceleration=np.zeros((3))
        for i in other_indices:
            acceleration += acceleration_between_2_points(m,i)
        return acceleration 

    # Storing acceleration, changes in velocity, and changes in displacement.
    acceleration_array=np.array([get_acceleration(i) for i in range(n)])
    final_velocity_array=velocity+acceleration_array*t
    displacement_array=velocity*t+0.5*acceleration_array*t**2
    
    # Updating particle positions and velocity.
    particles[:,1:4] += displacement_array
    particles[:,4:7] = final_velocity_array 
    
    # Return the particles with their updated positions and velocity.
    return particles 

# 3. Run the simulation and store position data
# 4. Plot the movement of the particles

def plot_n_body(particles, timestep, time):
# particles: a 2D list storing all the infomation of the particles 
# time: total length of time that pass 
    data_list=[]
    n=len(particles)
    iteration_no=int(time/timestep)

    # 1: data generation, expected output is data_list, a 3D array 
    for i in range(iteration_no):
        # 1.1: storing particle information 
        data_list.append(particles[:,1:4].copy())
        # 1.2: update particle information 
        particles=update_particles(particles, timestep)
    data_list=np.array(data_list)

    # 2: plotting 
    ax = plt.axes(projection='3d')
    for i in range(n):
        x=data_list[:,i,0]
        y=data_list[:,i,1]
        z=data_list[:,i,2]
        ax.plot3D(x, y, z)
    plt.show()
    
# Plot the movement of the particles
plot_n_body(particles, 0.01, 20)

We observe that both the particles undergo some funky movements when they run into each other, which is about as expected.

# The dam break model
Now that we've roughly seen how SPH works (not exactly, but a similar vein of thought), lets actually apply SPH to a toy problem. The [dam break problem](https://www.worldscientific.com/doi/pdf/10.1142/9789813223882_0001) is a classic problem in fluid dynamics that, as the name suggests, simulates what happens when a dam is suddenly broken. In the dam break problem, a body of water is initially contained in an area, and the simulation starts when the walls containing the body of water are removed (the dam breaking).

The dam break problem has been [written extensively on](https://pysph.readthedocs.io/en/latest/design/overview.html?highlight=dam%20break#the-dam-break-problem), and is therefore perfect for this notebook. This is because we know exactly what to expect when modelling, allowing us to validate our model. Thankfully, there are commercial solutions available such as the [`PySPH` package](https://pysph.readthedocs.io) which comes with nearly 100 inbuilt test cases, including the dam break problem.

`PySPH` has been undergoing development for the [almost fifteen years now](https://pysph.readthedocs.io/en/latest/overview.html#history), and is therefore far more optimized than anything we can write in 13 weeks (assuming we started this project in Week 1, which we did not). We therefore read the [rubrics](https://sps.nus.edu.sg/sp2273/assessment.html#rubric-for-presentation), which says that

> Suitable packages and functions are appropriately used without reinventing the wheel

and we therefore happily import the `PySPH` package to demonstrate how SPH is applied in Python.

**WARNING: THE NEXT CODE BLOCK REQUIRES THE `PySPH` PACKAGE, WHICH IS NOT A STANDARD INSTALL IN SP2273. IMPORT THIS PACKAGE BEFORE RUNNING THE NEXT CODE BLOCK.**

In [ ]:
%matplotlib ipympl

from matplotlib import pyplot as plt
from pysph.tools.ipy_viewer import Viewer2D

viewer = Viewer2D('db2d_output')
viewer.interactive_plot()

# Conclusion
To summarize, our goal in this notebook has been to demonstrate the use of Python to implement SPH for modelling flud dynamics. We began by acknowledging the challenges in finding exact solutions to the equations used to model fluid dynamics, which has lead us to investigate numerical methods to approximate solutions.

After a long and treacherous journey, we realized that nothing we can accomplish in six short weeks can match up to fifteen years of active development, and we import the `PySPH` package. In this notebook, we've investigated two toy examples to demonstrate that numerical methods can and do work, and we leave further work to the interested reader. Have fun!

# References
All our references have been consistently hyperlinked above, but here is a compilation of the links and where they go to for your convenience.

## Sources
- [Fluid dynamics (Wikipedia)](https://en.wikipedia.org/wiki/Fluid_dynamics)
- [Applications to biomedical engineering](https://pubmed.ncbi.nlm.nih.gov/34476751/)
- [Applications to disaster protection](https://link.springer.com/chapter/10.1007/978-3-211-69356-8_1)
- [Applications to industrial design](https://www.dynamiq-eng.co.uk/pages/Fluid_mechanics_dynamics.php)
- [Applications to astrophysics](https://arxiv.org/abs/1109.2219)
- [Fluid mechanics (Wikipedia)](https://en.wikipedia.org/wiki/Fluid_mechanics)
- [$Bi_4O_4SeCl_2$, the worst thermal conductor in the world](https://www.sciencealert.com/sustainability-win-scientists-make-material-with-the-lowest-thermal-conductivity-yet)
- [Continuum assumption](http://pillars.che.pitt.edu/student/slide.cgi?course_id=10&slide_id=5.0)
- [Navier Stokes](https://en.wikipedia.org/wiki/Navier–Stokes_existence_and_smoothness)
- [What is computational fluid dynamics](https://www.simscale.com/docs/simwiki/cfd-computational-fluid-dynamics/what-is-cfd-computational-fluid-dynamics/)
- [Dan Price and SPH](https://arxiv.org/abs/1111.1259)
- [Gaussian kernel](https://pages.stat.wisc.edu/~mchung/teaching/MIA/reading/diffusion.gaussian.kernel.pdf.pdf)
- [$n$-body problem](https://en.wikipedia.org/wiki/N-body_problem)
- [Dam break problem](https://www.worldscientific.com/doi/pdf/10.1142/9789813223882_0001)
- [Dam break explained](https://pysph.readthedocs.io/en/latest/design/overview.html?highlight=dam%20break#the-dam-break-problem)
- [PySPH package](https://pysph.readthedocs.io/en/latest/)
- [PySPH history](https://pysph.readthedocs.io/en/latest/overview.html#history)
- [SP2273 presentation rubrics](https://sps.nus.edu.sg/sp2273/assessment.html#rubric-for-presentation)

## Images
- [Bowl of water (Shutterstock)](https://www.shutterstock.com/image-vector/water-bowl-illustration-vector-cartoon-260nw-2210641359.jpg)
- [Navier Stokes equations (NASA)](https://www.grc.nasa.gov/www/k-12/airplane/Images/nseqs.gif)
- [Gaussian kernel](http://www.imagemet.com/WebHelp6/Content/Images/Gauss3D.png)

# Appendices
Here are some appendices with further details about different parts of the project.

## A1: Mathematics deep dive
In order to kickstart our simulation, we need to calculate the initial densities, pressures, and velocities of the particles in our system. Depending on how our simulation is set up, the positions of the particles may be randomly scattered, or they may be neatly ordered (as in the case of the dam break problem). From these initial positions, we may then calculate the initial density of the particles, as explained below.

To simplify our work, we denote the set of all the particles in our simulation by $X$. We now pick a single particle $a$ in $X$, and demonstrate how we can calculate the initial density, pressure, and velocity of particle $a$. In order to calculate the initial values for every particle in the system, we will then iterate through $X$, and repeat the calculations for each particle.

Firstly, the initial density for our particle $a$ is [given](https://pmocz.github.io/manuscripts/pmocz_sph.pdf) using the formula 

$$\rho_a=\sum_{b\in X-\{a\}} m_b \cdot \nabla W(\mathbf{r}_a - \mathbf{r}_b;h),$$

where $m_b$ refers to the mass of particle $b$, and $\mathbf{r}_i$ refers to the position of particle $i$. The formula $|\mathbf{r}_a - \mathbf{r}_b|$ therefore refers to the distance between particle $a$ and particle $b$, and the constant $h$ refers is known as the _smoothing length_ of the function $W$. By default, we set the smoothing length to $1$. This means that for any particle $b$ that is more than $1$ unit of distance away from particle $a$, we have $\nabla W(\mathbf{r}_a - \mathbf{r}_b ; h)=0$.

The term $\nabla W(r,h)$ is [defined](https://pmocz.github.io/manuscripts/pmocz_sph.pdf) as

$$\nabla W(r,h) = -\frac{2e^{\frac{-|r|^2}{h^2}} \cdot r} {h^{2+d} \cdot \pi^{\frac{d}{2}}},$$ 

where $d$ refers to the number of dimensions of the simulation. Since we're working in three-dimensional space, we have $d=3$. After calculating the initial density $rho_a$ of particle $a$, we then calculate the pressure $P_a$ of particle $a$ using the [following equation](https://iopscience.iop.org/article/10.1088/1742-6596/1985/1/012003/pdf):  

$$P(\rho) = P_0((\frac{\rho}{\rho_0})^\gamma)-1).$$

For fluids, $\gamma=7$ is often used. However, I have decided to subtract $0.5$ from the initial density, the reasons for which will be explained later. [$P_0$ is calculated as follows](https://iopscience.iop.org/article/10.1088/1742-6596/1985/1/012003/pdf):  

$$P_0 = \frac{100 \rho_0 v_{max}^2} {\gamma}$$  

After calculation of pressure and density, we can employ the following formula to [calculate acceleration](https://pmocz.github.io/manuscripts/pmocz_sph.pdf):

$$ \frac {d v_a}{dt} = - \sum_{b, b \neq a}^N m_b \left(\frac {P_a}{\rho^2_a} + \frac {P_b}{\rho^2_b} \right) \cdot \nabla W(r_a - r_b ; h) + B_a,$$

where $B_a$ is the acceleration of particle $a$ due to external forces (such as gravity acting on the entire system). However, in the case of the dam break problem, there are no external forces, and we have $B_a=0$. Lastly, we can then calculate the velocity of particle $a$, denoted $v_a$, as

$$ v_a = \frac {a_at^2}{2} + u_a,$$

where $u_a$ is the previous velocity. Put together, these yield a set of equations that allows us to update the relevant information about a particle $a$, based on the positions, velocities, accelerations, densities, and pressures of every other particle in the system.

## A2: Amateur implementations of SPH
Unlike the authors of the `PySPH` package, we do not have fifteen years of experience optimizing Python for doing complex calculations. However, here is one possible way of implementing an SPH model in Python.

In [ ]:
import numpy as np
import random

class RandomPArray():
    """
    This class generates and handles with
    multiple particle calculations.
    
    ...
    
    Attributes
    ----------
    g : int
        Seed for pseudorandom number generator.
        
    lbound : int
        Lower bound on the coordinates for the
        particles in the array.
    
    ubound: int
        Upper bound on the coordinates for the
        particles in the array.
        
    particles : list
        List of Particle objects in the simulation.
        
    Methods
    -------
    __init__(n, dims, seed)
        n : number of particles in the simulation
        dims: upper and lower bound of coordinates for the
              simulation
        seed: seed for pseudorandom number generator used
              to initialize the particle positions.
    
    calc_density()
        Calculates density of each individual particle
        and updates them.
    
    calc_accel()
        Calculates acceleration of each individual
        particle and updates them.
    
    run(duration)
        Updates the velocities, densities, and pressures of
        all particles in the simulation. Currently hardcoded to have
        a timestep of 0.01 seconds.
        
    get_init_rho()
        Testing function
        
    getBounds()
        Returns lower and upper bounds of the simulation.
        
    __str__
        Returns position and direction vector
        of every particle in the system.
    """
    def __init__(self, n, dims, seed):
        self.g = random.seed(seed)
        self.lbound = dims[0]
        self.ubound = dims[1]
        pPositions = []
        for i in range(n):
            pPositions.append(
                [random.randrange(self.lbound, self.ubound),
                 random.randrange(self.lbound, self.ubound),
                 random.randrange(self.lbound, self.ubound)])
        self.particles = [Particle(pPositions[i], [0, 0, 0]) for i in range(n)]
        for i in range(len(self.particles)):
            total = 0
            for j in range(len(self.particles)):
                # Particles cannot overlap
                if self.particles[i].getPos != self.particles[j].getPos:
                    total += Calculator(self.particles[i], self.particles[j]).dGausKern() * self.particles[j].m
            # Gamma is hardcoded as 7
            # Maximum velocity here is 10
            self.particles[i].initrho = total - 0.5
            self.particles[i].initPre = (100 * self.particles[i].initrho * (10**2))/7 
        
    def __str__(self):
        return f"Particles: Position {str([i.getPos() for i in self.particles])} and direction {str([i.getDir() for i in self.particles])}"
    
    def get_init_rho(self):
        for i in self.particles:
            print(i.initrho)
        return "End"
    
    def getBounds(self):
        return [self.lbound, self.ubound]
    
    def calc_density(self):

        for i in range(len(self.particles)):
            total = 0
            for j in range(len(self.particles)):
                if self.particles[i].getPos != self.particles[j].getPos: #Because particles cannot overlap
                    total += Calculator(self.particles[i], self.particles[j]).dGausKern() * self.particles[j].m #Multiply mass
            self.particles[i].rho = total
            print("Calculated density of " + str(i) + " is " +str(self.particles[i].rho))

    
    def calc_accel(self):
        """
        Checkpoint 3 of whee here.
        Refer to markdown for implementation
        Note that I am SUBTRACTING from its previous value.
        """
        for i in range(len(self.particles)):
            total = 0
            for j in range(len(self.particles)):
                if self.particles[i].getPos != self.particles[j].getPos: #Because particles cannot overlap
                    total += Calculator(self.particles[i], self.particles[j]).dGausKern() \
                    * ( (self.particles[i].Pre / (self.particles[i].rho**2)) \
                       + (self.particles[i].Pre / (self.particles[j].rho**2)) ) \
                    * self.particles[j].m
                    
            self.particles[i].a -= total
            print("Calculated acceleration of " + str(i) + " is " +str(self.particles[i].a))
        
    def run(self, duration):
        while duration > 0:
            for i in range(len(self.particles)):
                self.particles[i].move()
                
            self.calc_density()
            
            for i in range(len(self.particles)):
                self.particles[i].calc_Pressure()
                
            self.calc_accel()
            
            for i in range(len(self.particles)):
                self.particles[i].calc_veloc()
                print("Calculated position of " + str(i) + " is " +str(self.particles[i].p))
            duration -= 0.01
            
class Particle():
    """
    This class handles individual particles, and does
    calculations only requiring individual particles.
    
    ...
    
    Attributes
    ----------
    g : int
        Seed for pseudorandom number generator.
        
    lbound : int
        Lower bound on the coordinates for the
        particles in the array.
    
    ubound: int
        Upper bound on the coordinates for the
        particles in the array.
        
    pPosparticles : list
        List of Particle objects in the simulation.
        
    p : list
        List with three elements, representing the
        position of the particle.
        
    d : list
        List with three elements, representing the
        direction vector of the particle.
        
    m : float
        Mass of the particle.
        
    dt : float
        Timestep of the system.
        
    t : float
        Total time of the system.
        
    rho: float
        Density of the particle.
        
    initrho : float
        Initial density of the particle.
        
    Pre : float
        Pressure acting on the particle.
        
    initPre : float
        Initial pressure acting on the particle.
        
    a : list
        Acceleration vector of the particle.
        
    vmax : float
        Maximum velocity of the particle.

    Methods
    -------
    __init__(position, direction)
        Initializes a new particle given a position
        and direction vector.
            
    __str__()
        Returns current position and direction vector.
            
    getPos()
        Returns current position.
            
    getDir()
        Returns current direction.
        
    getMagnitude()
        Returns the distance of the particle from the origin.
        
    move()
        Updates position of particle based on its
        direction vector. If it has no direction
        vector, returns current position of the particle.
            
    calc_Pressure()
        Calculates pressure of the particle.
            
    calc_veloc()
        Calculates velocity of the particle.
        
    newPos(newPos)
        Updates the position of the particle with
        the value newPos.
    """
    def __init__(self, position, direction):
        self.p = position
        self.d = direction
        self.m = 1
        self.dt = 0.01
        self.t = 0
        self.rho = 0
        self.initrho = 1
        self.Pre = 0
        self.initPre = 0
        self.a = 0
        self.vmax = 10
    
    def __str__(self):
        return f"Current position: {self.p}. Current direction: {self.d}." #Prevents you from breaking code
    
    def getPos(self):
        return self.p
    
    def getDir(self):
        return self.d

    def getMagnitude(self):
        return np.linalg.norm(self.p)
    
    def move(self):
        temp_pos = self.getPos().copy()
        temp_dir = self.getDir().copy()
        if self.getDir() != [0,0,0]:
            for i in range(3):
                temp_pos[i] += temp_dir[i] * self.dt
        self.t += self.dt
        return self.newPos(temp_pos)
    
    def calc_Pressure(self):
        self.Pre = self.initPre * (((self.rho/self.initrho)**7)-1)
        print("Current pressure: " + str(self.Pre))
        
    def calc_veloc(self):
        for i in range(3):
            potential_speed = (self.a * (self.t**2))/2
            if self.d[i] + potential_speed > self.vmax:
                self.d[i] = 10
            else:
                self.d[i] += potential_speed
        print("Current velocity: " + str(self.d))
    
    def newPos(self, newPos):
        self.p = newPos

class Calculator():
    """
    A pairwise calculator which takes in two Particle objects and
    calculates thesmoothing kernel for them. This calculator
    does not check if the particles are identical.
    
    ...
    
    Attributes
    ----------
    p1 : Particle
        Main Particle object.
        
    p2: Particle
        Other particle for p1 to be compared
        with.
        
    Methods
    -------
    __init__(p1, p2)
        Initializes the Calculator object with
        two particles.
        
    dGausKern(h)
        Calculates the gradient of the gaussian
        kernel, with a default smoothing length
        scale h = 1.
    """
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
    
    def dGausKern(self ,h=1):
        p1_mag = self.p1.getMagnitude()
        p2_mag = self.p2.getMagnitude()
        delta_mag = p1_mag - p2_mag
        dGaus = -((2*np.e**(-(delta_mag**2) / (h**2)))*(delta_mag)) / ((h**5)*(np.pi*(3/2)))
        return dGaus

In [ ]:
# Generates a sample array with 10 particles in the range [0,3].
sampleArray = RandomPArray(10, [0,3], 4)

# Runs the simulation for 1 second, with a timestep of 0.01 seconds.
sampleArray.run(1)